In [3]:
import sys
sys.path.append('..')
from dataloader.datamodule_ecog2ph import ECoGPhonemeDataset, ECoGPhonemeDataModule
import numpy as np
import torch
from utils.utils import load_cfg_and_ckpt_path
from model.phoneme_ctc import SpeechModel
import time
import tqdm
import torchaudio
from pathlib import Path
import re
import string
regex = re.compile('[%s]' % re.escape(string.punctuation))


/home/cheoljun/anaconda3/envs/transformer/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:

cfg,_ = load_cfg_and_ckpt_path('../outputs/2023-10-02/08-13-46/phoneme_ctc_tm1k')

datamodule = ECoGPhonemeDataModule(**cfg['data'])


ecog_files =([f for f,_ in datamodule._load_data('train')]+
            [f for f,_ in datamodule._load_data('val')]+
             [f for f,_ in datamodule._load_data('test')])

texts = ([t for _,t in datamodule._load_data('train')]+
            [t for _,t in datamodule._load_data('val')]+
             [t for _,t in datamodule._load_data('test')])

#ecog_file
save_dir = Path('/data/cheoljun/b3_features/phoneme_posterior')/cfg['data']['paradigm']


In [5]:
lens=[]
word_lens=[]
for ecog_file,text in tqdm.tqdm(zip(ecog_files,texts)):
    save_path = ecog_file.stem.split('_')[-1]
    save_path = save_dir/f'{save_path}.npy'
    phpos = np.load(save_path)
    non_blank = np.nonzero((phpos[:,:2].sum(-1)<0.5)*1.0)[0]
    start, end = non_blank[0],non_blank[-1]
    word_lens.append(len(text.split(' ')))
    lens.append(end-start+1)

9887it [00:03, 2878.29it/s]


In [54]:
len(phpos)*sr

9.0

In [ ]:
fileidx2textidx=np.load('/data/cheoljun/b3_audio_scale-2/tm1k/file2textindex.npy',allow_pickle=True)[()]

In [32]:
sr = 4*6/200

In [49]:
np.sum(lens)/np.sum(word_lens)*sr

0.7742435424354244

In [57]:
wpm = np.sum(word_lens)/(np.sum(lens)*sr/60)

In [58]:
wpm

77.49499571060909

In [52]:
130/wpm

1.677527675276753

In [47]:
sample_texts=np.array(['What class did you say that was?', 'Sometimes I just need a hug.',
       'I keep telling myself that.', 'Nothing what could I do?',
       'About my blood work?', 'Never mind, just say it.',
       'Exactly two hundred dollars a pop.',
       'Alright what did you think?', 'How do you know this place?',
       'I know I called them.', 'First thing that comes to your mind?',
       'Where do I put this!?', 'And a missing man.',
       'But you knew about this.', 'What is the point?',
       'What is it, dad?', 'I cannot tell a lie.',
       'I wonder where they went.', 'Which one is for the car?',
       'You had one guy like that?', 'Where are they? Do you see them?',
       'So it would seem.', 'The kids are in here!', 'Can we come in?',
       'I wish we could start over', 'Wait, I wanted her to do it.',
       'I said: I found a room.', 'No, that door over there.',
       'Keep warm, dear let me hear from you.',
       'Who was that on the phone?', 'And be right back.',
       'I know how that can be.', 'Whatever I can get away with.',
       'I want you to do this work.', 'Give it back to me.',
       'Get them in the lab!', 'If it works you mean.',
       'Think about it okay?', 'Your feet are cold.',
       'Another kind of game?', 'Does it have a television?',
       'If I change my mind.', 'Can you do it?',
       'No, I left it in his office.', 'Because she is my friend.',
       'I want you to do something for me.', 'Thanks for the coffee.',
       'Have you pulled yourself together?', 'Same weight I always was.',
       'Did you mail then?', 'Put in a quarter.',
       'I want to check something hold my feet.', 'Why did you quit?',
       'And the one driving the other car?', 'My father was a sick man.',
       'She left a note.', 'He wants a lot of money.',
       'But what about lunch?', 'Do you see them yet?',
       'I believe in you.', 'There goes your ride.', 'She talks to me.',
       'He was looking out for you.', 'Does it have a human name?',
       'This is not real life', 'You like ball games?',
       'Take another look at these two.',
       'And still not a thing to watch.', 'We got to get out of here.',
       'Were you always this sick?', 'I done it again.',
       'Did you really say that?', 'But where did you finally end up?',
       'Are you out of your mind?', 'When did you see him last?',
       'Does she love the cat?', 'Go ahead you can say it.',
       'Not until he has the money!', 'They get hair all over the place.',
       'Want to come out?', 'You can count on it.',
       'Who was that? Somebody out there?', 'Anyway if you need me.',
       'What do you mean there is no family?',
       'Wish I could say the same.', 'How much more a month?',
       'What do your friends call you?', 'When I was very young.',
       'Can anything be done for him?', 'I will start with you.',
       'Mom says to clean up the pool.', 'How far from here?',
       'Look, either come in or stay out!', 'Do you have everything?',
       'I like your hat.', 'Why are you here, then?',
       'What took you so long?', 'How bad is it for you?',
       'Can I give her a message?', 'Well, when do you get paid?'])

sample_texts_new=[]
for text in sample_texts:
    text = regex.sub('', text.lower())
    text = ' '.join([t for t in text.split(' ') if t != ''])
    sample_texts_new.append(text)
sample_texts=sample_texts_new

In [48]:
lens=[]
word_lens=[]
for ecog_file,text in tqdm.tqdm(zip(ecog_files,texts)):
    if text not in sample_texts:
        continue
    save_path = ecog_file.stem.split('_')[-1]
    save_path = save_dir/f'{save_path}.npy'
    phpos = np.load(save_path)
    non_blank = np.nonzero((phpos[:,:2].sum(-1)<0.5)*1.0)[0]
    start, end = non_blank[0],non_blank[-1]
    word_lens.append(len(text.split(' ')))
    lens.append(end-start+1)

9887it [00:00, 219798.08it/s]
